### create and save bertopic model and new csv

In [ ]:
import pandas as pd
df = pd.read_csv('text_topics.csv')

for big bert model remove messages with more than 250 words

In [ ]:
df_shorter = df[df['word_count'] <= 250]
print(len(df))
print(len(df_shorter))

In [ ]:
from sentence_transformers import SentenceTransformer
import nltk
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer
from bertopic import BERTopic

# Prepare NLP and clustering tools
nltk.download('stopwords')
stopwords_combined = stopwords.words('german') + stopwords.words('french') + \
                      stopwords.words('spanish') + stopwords.words('english')

model = SentenceTransformer("all-MiniLM-L6-v2")
vectorizer = CountVectorizer(stop_words=stopwords_combined)

# Assuming 'df' is your entire dataset
documents = df_shorter['text_clean'].tolist()
document_vectors = model.encode(documents, show_progress_bar=True)

# Create a single BERTopic model for the entire dataset
topic_model = BERTopic(embedding_model=model, vectorizer_model=vectorizer, verbose=True)
topics = topic_model.fit_transform(documents, document_vectors)[0]  # Get topic numbers

# Create a DataFrame from topic_assignments_list with 'message_id' and the new 'topic_big' column
topic_assignments_df = pd.DataFrame({'message_id': df_shorter['message_id'], 'topic_big': topics})

# Merge the new topic assignments with the original dataframe
# Assuming 'message_id' is a unique identifier for each row/document
df_new = pd.merge(df_shorter, topic_assignments_df, on='message_id', how='left')

print(f"BERTopic model created for the entire dataset with {len(documents)} documents. 'topic_big' column added to the DataFrame.")


In [ ]:
df_new.to_csv("text_topics_bigmodel.csv", index=False)

In [ ]:
num_duplicates = df_new.duplicated(subset='text_clean', keep='first').sum()

print(f"Number of duplicate entries in 'text_clean': {num_duplicates}")

In [ ]:
topic_model.save("good_bigmodel_bertopic", serialization="pickle")


In [ ]:
!pip freeze > requirements_bigtopic.txt

### analyse the topics

In [ ]:
import pandas as pd
df_new = pd.read_csv('text_topics_bigmodel.csv')

In [ ]:
from bertopic import BERTopic

In [ ]:
topic_model = BERTopic.load('good_bigmodel_bertopic')

In [ ]:
def find_and_print_similar_topics(word):
    similar_topics, similarity = topic_model.find_topics(word, top_n=5)
    topic_info_df = topic_model.get_topic_info()
    
    print(f"\nSimilar topics for '{word}':")
    for topic, similarity_score in zip(similar_topics, similarity):
        if topic == -1:
            print(f"  - Topic Number: -1, Name: Outliers/Noise, Similarity Score: {similarity_score}, Representation: N/A")
        else:
            name = topic_info_df[topic_info_df.Topic == topic]['Name'].values[0]
            representation = topic_info_df[topic_info_df.Topic == topic]['Representation'].values[0]
            print(f"  - Topic Number: {topic}, Name: {name}, Similarity Score: {similarity_score}, Representation: {representation}")


In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from IPython.display import display, HTML  # Import display and HTML for rendering HTML

# Adjust display options for pandas
pd.set_option('display.max_colwidth', None)

# Function definition
def analyze_topic(topic_model, topic_number):
    # Filter df for both topic number
    filtered_df = df_new[(df_new['topic_big'] == topic_number)]
    
    # Visualize the barchart for the topic
    try:
        fig = topic_model.visualize_barchart(topics=[topic_number], height=300, width=600)
        display(fig)  # Explicitly display the figure
    except Exception as e:
        print(f"An error occurred while generating the barchart: {e}")
    
    # Get and display the top 2 representative documents for the topic
    representative_docs = topic_model.get_representative_docs(topic=topic_number)
    print("\nTop 2 Representative Documents for Topic", topic_number)
    for doc in representative_docs[:2]:
        print("---")
        print(doc)
    
    # Get and display the number of documents assigned to the topic
    topic_count = len(filtered_df)
    doc_count_html = f"Number of <b>Documents Assigned</b> to Topic {topic_number} : <b>{topic_count}<b>"
    display(HTML(doc_count_html))

    # Print the number of unique 'channel_name' values for the topic_number within the word count range
    unique_channels = filtered_df['channel_name'].nunique()
    unique_channels_html = f"Number of <b>Unique Channels</b> for Topic {topic_number} : <b>{unique_channels}<b>"
    display(HTML(unique_channels_html))

    # Look at some random documents for a better idea of the topic
    if not filtered_df.empty:
        random_samples = filtered_df['text_clean'].sample(4, random_state=np.random.RandomState())
        print(f"\n4 Random Documents for Topic {topic_number} :")
        for doc in random_samples:
            print("---")
            print(doc)
    else:
        print(f"No documents found for Topic {topic_number}.")

In [ ]:
find_and_print_similar_topics("lingreen")

keywords = ["wahl", "politik", "elite", "regierung", "betrug", "volk", "leute", "unterdrückung", "staat", 
            "gewalt", "machtbegrenzung", "schwarmintelligenz", "freiheit", "basis", "partei", "altparteien", 
            "system", "plandemie", "basisdemokratie", "johanna", "findeisen", "reuss", "demokratie", "rentnerputsch", "wissenschaft", 
            "reichsbürger", "experte", "volksverrat", "lockdown", "impfzwang", "bundesvorstand", "parteiengesetz", "volksentscheid", "gender", "konsens", "team", "spaltung", "rockefeller","geheim", "gates", "diktatur", "information", "weltgesundheitsorganisation", "rki", "pandemievertrag", "russland", "unterwandert", "corona", "bauern", "genozid","dialog", "diskurs", "antifa", "common sense","vernunft", "meinung", "fakt", "verstehen", "forschung", "universität", "wahrheit", "statistik", "smartcity"]

 |broad topic | number | name |thoughts |
 |:---------- | :----- |:---- |:---- |
| covid|  5 | masken  |
| politik_issues|  6 | nato  |
| politik_issues|  7 | twitter  |
| politik_issues|  11 | elektroautos   |
| politik_issues|  12 | nordstream   |
| politik_issues|  381 | strompreise   |
| covid|  13 | impfung | 
| covid|  33 | impfung | 
| covid|  16 | covid_hysterie  | plandemie, hysterie|
| rechts|  17 |trump|
|verschworung|18| katastrophe_viruskrieg| covid-specific conspiracy|
| gegen_links|  20 |antifa|
| politik_kritik|  23 | politik_polizisten  | hier als 'anti politdarsteller'|
| russland|  24 | putin |
| russland|  47 | russland_generell |
| covid|  25 | impfung_mrna | 
| demokratie|  1047 | keine_echte_wahl | 
| demokratie|  3213 | wahlen_problem | all sorts of things, random topic on elections, negative| 
| politik_kritik|  715 | faschismus | 
| gegenwissen|  674 | verstehen_verstand | verstnad verloren, leute verstehen|
| aktivismus_regional|  325 | stuendenten_aktivismus |
| aktivismus_regional|  1016 | stuendenten_aktivismus_info | mehr info
| anti_elite|  5305 |harvard_elite|
| anti_elite|  586 |anti_elite_generell| 'gibt es sie doch, die eliten?'|
| demokratie|  132 |demokratie_generell|
| verschworung|  5264 |johnshopkins_elite|
| verschworung|  2032 |illuminati|
| verschworung|  3843 |demokratie_satan| verschwörungstheorien dass demokratie satan ist
| epistemisierung|  1198 |wahrheitsdiskurs_relativ_positivistisch|
| zensur|  2297 |wahrheitsministerium|
| zensur|  3151 |wahrheitsministerium|
| zensur|  4199 |panikpresse|
| politik_kritik|  1145 | quarantäne_gegen_menschen  |
| politik_kritik|  889 | menscheit_wachtauf_wahrheit  |
| verschworung|  751 | kriegsvorbereitung  |
| verschworung|  616 | internationale_organisationen_verschworung  |
| politik_kritik|  2405 | parteienstaat  |
| politik_kritik|  116 | bundesinstitutionen  |
| politik_kritik|  2613 | bundestagswahl2021_  |
| politik_kritik|  6398 | politiker_spalten  |
| politik_kritik|  1775 | jens_spahn_kritik |
| politik_kritik|  2327 | jens_spahn_kritik |
| politik_kritik|  9074 | jens_spahn_kritik |
| politik_kritik|  970 | regierung_staatsfeinde |
| politik_kritik|  821 | verfassungsschutz |
| politik_kritik|  225 | systemkritik | einfach globale systemkritik|
| demokratie|  6334 | gegen_gewalt|
| covid|  6279 | impfindustrie | 
| verschworung|  664 | alles_ist_ein_plan  |
| covid|  1754 | plandemie  |
| covid|  34 | lockdown  |
| covid|  2744 | impfzwang  |
| covid|  470 | impfzwang_2  |
| covid|  101 | pandemie_generell  |
| covid|  467 | pandemievertrag  |
| covid|  446 | genesenenstatus  |
| zensur|  1022 | dsa_kritik_zensur   |
| zensur|  1022 | dsa_kritik_zensur   |
| politik_kritik|  533 | volk_ist_souverän  |
| politik_kritik|  495 | bauernproteste  |
| politik_kritik|  5101 | bauernproteste  |
| ???|  81 | wissenschaft_botschaft_gewerkschaft  |
| gegenwissen|  1127 | gegenexperten  |
| gegenwissen | 1621 | statistik  |
| politik_kritik | 5344 | völkerrecht  |
| aktivismus_regional | 796 | demo_magdeburg  |
|aktivismus_regional | 1392 | demo_hamburg |
|aktivismus_regional | 2929 | demo_rostock |
|aktivismus_regional | 3040 | demo_berlin_august |
|aktivismus_regional | 1341 | demo_berlin|
|aktivismus_regional | 8362 |demo_donauwörth|
|aktivismus_regional | 387 | spaziergange_generell|
|diebasis | 1358 | sachsen_diebasis_demo|
|diebasis | 8092 | diebasisinterviews|
|politik_issues|  904 | smart_cities  |
|politik_issues|  1400 | e_id_generell |
|verschworung|  7055 | e_id_verschworung |
| rechts|  2925 |bevolkerungsaustausch|
|verschworung|  604 | rockefeller |
|verschworung|  5360 | machtzirkel_geheimplan |
|verschworung|  579 | freimaurer |
|verschworung|  123 | gates |
|verschworung|  1305 | gates |
|verschworung|  1261 | gates |
|verschworung|  4005 | informationskrieg |
|verschworung|  1305 | wef |
|verschworung|  941 | reichsbürger_generell |
|verschworung|  9422 | reichsbürger_razzia |
|verschworung|  3348 | reichsbürger_razzia |
| gegen_links|  249 |volksverpetzer_kritik|
| gegen_links|  113 |antifeminismus_gendersprache|
| gegen_links|  1361 |antifeminismus_gender_skandalinfo|
| gegen_links|  580 |antifeminismus_frühkindsexualisierung|
| gegen_links|  1361 |antifeminismus_gender_skandalinfo|
| politik_kritik|  356 | antisemitismus_generell  |
| politik_kritik|  257 | antifa  |
| gegenwissen|  237 | gegenexperte_daniele_ganser  |
| gegenwissen|  4715 | wissenschaft_widerstand  |
| gegenwissen|  587 | gegenexperte_gunnar_kaiser |
| gegenwissen|  1282 | inside_worldbank_reset  |
| politik_kritik|  183 | verschwörungstheoretiker  | 1 wir sind vtheoretiker; 2 kritik von dem label
| politik_kritik|  821 | verfassungsschutz  |
| politik_kritik|  1873 | meinungsfreiheit  |
| politik_issues|  1068 | myanmar_putsch  |
| gegenwissen|  747 | faktenchecker_kritik  | die sind gekauft
| gegenwissen|  1282 | inside_worldbank_reset  |
|gegenwissen | 1205 | gegenexperte_martin_schwab|
|diebasis | 7120 | diebasisinterviews|
|diebasis | 8661 | diebasis_lifestyle|
|diebasis | 3083 | diebasis_einbringen|
|diebasis | 3935 | diebasis_info?|
|diebasis | 7107 | diebasis_lifestream|
|diebasis | 4480 | diebasis_machtbegrenzung_spende|
|diebasis | 6933 | diebasis_gesundheit|
|politik_kritik | 4638 | gegenparteien|
|diebasis | 2062 | diebasis_spende|
|diebasis | 5709 | diebasis_aktivebeteiligung|
|diebasis | 1931 | diebasis_hashtags|
|diebasis | 2302 | diebasis_doppelspitze|
|diebasis | 8786 | diebasis_mitentscheiden|
|diebasis | 7935 | diebasis_vorstellung|
|diebasis | 5188 | diebasis_hashtags|
|diebasis | 1757 | diebasis_hashtags|
|diebasis | 3436 | diebasis_shortmessages|
|diebasis | 7965 | diebasis_shortmessages|
|diebasis | 5226 | diebasis_akademie
|diebasis | 6665 | diebasis_podcast|
|diebasis | 4493 | diebasisradio|
|diebasis | 7209 | diebasis_generell|
|politik_issues|  27 | hamas_gaza  |
|politik_issues|  28 | europa_generell  |
|politik_issues|  31 | leopardpanzer  |
|politik_issues|  32 | taliban_afghanistan  |
|politik_issues|  33 | impfung  |
|politik_issues|  40 | julian_assange_wikileaks  |
|gegen_links|  41 | klimawandel  |
|gegen_links|  43 | habeck  |
|gegen_links|  44 | co2_emissionen  |
|rechts|45| migration |
|politik_issues|  48 | inflation  |
|frames|  50 | diskursframe_virus  |
|frames|  51 | gegenlinkspartei  |
|verschworung|  52 | pharmaindustrie|
|politik_issues|  54 | gas_lieferung_gazprom|
|politik_issues|  55 | jeffrey_epstein| anti elite, verschworung auch mit dabei
|politik_issues|  61 | israel_gaza|
|politik_issues|  69 | bank_suisse|
|verschworung|  70 | immunsystem|
|verschworung|  71 | merkel|
|verschworung|  70 | immunsystem|
|russland|  74 | russland_diskriminierung|
| aktivismus_regional | 75 | demo_leipzig |
|politik_issues|  80 | trucker_protests_canada|
|frames|  87 | krankenhäuser|
|frames|  90 | protest_generell|
|frames|  87 | krankenhäuser|
|demokratie|  96 | bürgerdialog_kritik|
|politik_issues|  98 | iran|
|politik_issues|  99 | facebook|

In [ ]:
analyze_topic(topic_model, 99)

In [ ]:
# Assuming topic_model is your BERTopic model
topic_freq = topic_model.get_topic_info()

# Filter for the first 100 topics, excluding the -1 (outlier) topic if you wish
filtered_topic_freq = topic_freq.head(101)  # Includes the -1 topic by default

# Visualize the bar chart
topic_model.visualize_barchart(top_n_topics=100, topics=filtered_topic_freq['Topic'].tolist())


In [ ]:
# Assuming you have the markdown table as a string, you will first convert it to a list of dictionaries
import pandas as pd
import re

# Sample markdown table text
markdown_table = """
|broad topic | number | name |thoughts |
 |:---------- | :----- |:---- |:---- |
| covid|  5 | masken  |
| politik_issues|  6 | nato  |
| politik_issues|  7 | twitter  |
| politik_issues|  11 | elektroautos   |
| politik_issues|  12 | nordstream   |
| politik_issues|  381 | strompreise   |
| covid|  13 | impfung | 
| covid|  33 | impfung | 
| covid|  16 | covid_hysterie  | plandemie, hysterie|
| rechts|  17 |trump|
|verschworung|18| katastrophe_viruskrieg| covid-specific conspiracy|
| gegen_links|  20 |antifa|
| politik_kritik|  23 | politik_polizisten  | hier als 'anti politdarsteller'|
| russland|  24 | putin |
| russland|  47 | russland_generell |
| covid|  25 | impfung_mrna | 
| demokratie|  1047 | keine_echte_wahl | 
| demokratie|  3213 | wahlen_problem | all sorts of things, random topic on elections, negative| 
| politik_kritik|  715 | faschismus | 
| gegenwissen|  674 | verstehen_verstand | verstnad verloren, leute verstehen|
| aktivismus_regional|  325 | stuendenten_aktivismus |
| aktivismus_regional|  1016 | stuendenten_aktivismus_info | mehr info
| anti_elite|  5305 |harvard_elite|
| anti_elite|  586 |anti_elite_generell| 'gibt es sie doch, die eliten?'|
| demokratie|  132 |demokratie_generell|
| verschworung|  5264 |johnshopkins_elite|
| verschworung|  2032 |illuminati|
| verschworung|  3843 |demokratie_satan| verschwörungstheorien dass demokratie satan ist
| epistemisierung|  1198 |wahrheitsdiskurs_relativ_positivistisch|
| zensur|  2297 |wahrheitsministerium|
| zensur|  3151 |wahrheitsministerium|
| zensur|  4199 |panikpresse|
| politik_kritik|  1145 | quarantäne_gegen_menschen  |
| politik_kritik|  889 | menscheit_wachtauf_wahrheit  |
| verschworung|  751 | kriegsvorbereitung  |
| verschworung|  616 | internationale_organisationen_verschworung  |
| politik_kritik|  2405 | parteienstaat  |
| politik_kritik|  116 | bundesinstitutionen  |
| politik_kritik|  2613 | bundestagswahl2021_  |
| politik_kritik|  6398 | politiker_spalten  |
| politik_kritik|  1775 | jens_spahn_kritik |
| politik_kritik|  2327 | jens_spahn_kritik |
| politik_kritik|  9074 | jens_spahn_kritik |
| politik_kritik|  970 | regierung_staatsfeinde |
| politik_kritik|  821 | verfassungsschutz |
| politik_kritik|  225 | systemkritik | einfach globale systemkritik|
| demokratie|  6334 | gegen_gewalt|
| covid|  6279 | impfindustrie | 
| verschworung|  664 | alles_ist_ein_plan  |
| covid|  1754 | plandemie  |
| covid|  34 | lockdown  |
| covid|  2744 | impfzwang  |
| covid|  470 | impfzwang_2  |
| covid|  101 | pandemie_generell  |
| covid|  467 | pandemievertrag  |
| covid|  446 | genesenenstatus  |
| zensur|  1022 | dsa_kritik_zensur   |
| zensur|  1022 | dsa_kritik_zensur   |
| politik_kritik|  533 | volk_ist_souverän  |
| politik_kritik|  495 | bauernproteste  |
| politik_kritik|  5101 | bauernproteste  |
| ???|  81 | wissenschaft_botschaft_gewerkschaft  |
| gegenwissen|  1127 | gegenexperten  |
| gegenwissen | 1621 | statistik  |
| politik_kritik | 5344 | völkerrecht  |
| aktivismus_regional | 796 | demo_magdeburg  |
|aktivismus_regional | 1392 | demo_hamburg |
|aktivismus_regional | 2929 | demo_rostock |
|aktivismus_regional | 3040 | demo_berlin_august |
|aktivismus_regional | 1341 | demo_berlin|
|aktivismus_regional | 8362 |demo_donauwörth|
|aktivismus_regional | 387 | spaziergange_generell|
|diebasis | 1358 | sachsen_diebasis_demo|
|diebasis | 8092 | diebasisinterviews|
|politik_issues|  904 | smart_cities  |
|politik_issues|  1400 | e_id_generell |
|verschworung|  7055 | e_id_verschworung |
| rechts|  2925 |bevolkerungsaustausch|
|verschworung|  604 | rockefeller |
|verschworung|  5360 | machtzirkel_geheimplan |
|verschworung|  579 | freimaurer |
|verschworung|  123 | gates |
|verschworung|  1305 | gates |
|verschworung|  1261 | gates |
|verschworung|  4005 | informationskrieg |
|verschworung|  1305 | wef |
|verschworung|  941 | reichsbürger_generell |
|verschworung|  9422 | reichsbürger_razzia |
|verschworung|  3348 | reichsbürger_razzia |
| gegen_links|  249 |volksverpetzer_kritik|
| gegen_links|  113 |antifeminismus_gendersprache|
| gegen_links|  1361 |antifeminismus_gender_skandalinfo|
| gegen_links|  580 |antifeminismus_frühkindsexualisierung|
| gegen_links|  1361 |antifeminismus_gender_skandalinfo|
| politik_kritik|  356 | antisemitismus_generell  |
| politik_kritik|  257 | antifa  |
| gegenwissen|  237 | gegenexperte_daniele_ganser  |
| gegenwissen|  4715 | wissenschaft_widerstand  |
| gegenwissen|  587 | gegenexperte_gunnar_kaiser |
| gegenwissen|  1282 | inside_worldbank_reset  |
| politik_kritik|  183 | verschwörungstheoretiker  | 1 wir sind vtheoretiker; 2 kritik von dem label
| politik_kritik|  821 | verfassungsschutz  |
| politik_kritik|  1873 | meinungsfreiheit  |
| politik_issues|  1068 | myanmar_putsch  |
| gegenwissen|  747 | faktenchecker_kritik  | die sind gekauft
| gegenwissen|  1282 | inside_worldbank_reset  |
|gegenwissen | 1205 | gegenexperte_martin_schwab|
|diebasis | 7120 | diebasisinterviews|
|diebasis | 8661 | diebasis_lifestyle|
|diebasis | 3083 | diebasis_einbringen|
|diebasis | 3935 | diebasis_info?|
|diebasis | 7107 | diebasis_lifestream|
|diebasis | 4480 | diebasis_machtbegrenzung_spende|
|diebasis | 6933 | diebasis_gesundheit|
|politik_kritik | 4638 | gegenparteien|
|diebasis | 2062 | diebasis_spende|
|diebasis | 5709 | diebasis_aktivebeteiligung|
|diebasis | 1931 | diebasis_hashtags|
|diebasis | 2302 | diebasis_doppelspitze|
|diebasis | 8786 | diebasis_mitentscheiden|
|diebasis | 7935 | diebasis_vorstellung|
|diebasis | 5188 | diebasis_hashtags|
|diebasis | 1757 | diebasis_hashtags|
|diebasis | 3436 | diebasis_shortmessages|
|diebasis | 7965 | diebasis_shortmessages|
|diebasis | 5226 | diebasis_akademie
|diebasis | 6665 | diebasis_podcast|
|diebasis | 4493 | diebasisradio|
|diebasis | 7209 | diebasis_generell|
|politik_issues|  27 | hamas_gaza  |
|politik_issues|  28 | europa_generell  |
|politik_issues|  31 | leopardpanzer  |
|politik_issues|  32 | taliban_afghanistan  |
|politik_issues|  33 | impfung  |
|politik_issues|  40 | julian_assange_wikileaks  |
|gegen_links|  41 | klimawandel  |
|gegen_links|  43 | habeck  |
|gegen_links|  44 | co2_emissionen  |
|rechts|45| migration |
|politik_issues|  48 | inflation  |
|frames|  50 | diskursframe_virus  |
|frames|  51 | gegenlinkspartei  |
|verschworung|  52 | pharmaindustrie|
|politik_issues|  54 | gas_lieferung_gazprom|
|politik_issues|  55 | jeffrey_epstein| anti elite, verschworung auch mit dabei
|politik_issues|  61 | israel_gaza|
|politik_issues|  69 | bank_suisse|
|verschworung|  70 | immunsystem|
|verschworung|  71 | merkel|
|verschworung|  70 | immunsystem|
|russland|  74 | russland_diskriminierung|
| aktivismus_regional | 75 | demo_leipzig |
|politik_issues|  80 | trucker_protests_canada|
|frames|  87 | krankenhäuser|
|frames|  90 | protest_generell|
|frames|  87 | krankenhäuser|
|demokratie|  96 | bürgerdialog_kritik|
|politik_issues|  98 | iran|
|politik_issues|  99 | facebook|
"""

# Process the markdown table
lines = markdown_table.strip().split("\n")[2:]  # Skip header lines
data = []
for line in lines:
    # Split the line into columns based on "|"
    cols = [col.strip() for col in line.split("|") if col.strip()]
    # Create a dictionary for each row, assuming a fixed column structure
    if len(cols) >= 4:  # Ensure there are at least 4 columns to avoid index errors
        row = {
            "broad topic": cols[0],
            "number": cols[1],
            "name": cols[2],
            "thoughts": cols[3] if len(cols) == 4 else ""  # Add thoughts if present, else empty string
        }
        data.append(row)

# Convert the list of dictionaries to a DataFrame
bigtopics = pd.DataFrame(data)

# Convert DataFrame to CSV
#csv_topics = topics.to_csv(index=False)

In [ ]:
bigtopics

In [ ]:
bigtopics.to_csv('bigtopics.csv', index=False)

In [ ]:
print(df_new.columns.tolist())

In [ ]:
print(f"Data type of 'topic_big' in df_new: {df_new['topic_big'].dtype}")
print(f"Data type of 'number' in bigtopics: {bigtopics['number'].dtype}")


In [ ]:
df_new['topic_big'] = df_new['topic_big'].astype(int)
bigtopics['number'] = bigtopics['number'].astype(int)

# Now perform the merge
df_merged = pd.merge(df_new, bigtopics, left_on='topic_big', right_on='number', how='left')
print(df_merged.columns.tolist())

In [ ]:
mismatched_rows = df_merged[~df_merged['number'].isna() & (df_merged['topic_big'] != df_merged['number'])]

# Display the number of mismatched rows
print(f"Number of mismatched rows: {len(mismatched_rows)}")

In [ ]:
bins = [0, 10, 20, 50, 80, 100, 150, 200, 250, float('inf')]
labels = ['<10', '10-20', '21-50', '51-80', '81-100', '101-150', '151-200', '201-250', '>250']

# Use pd.cut to assign word count ranges
df_merged['word_count_range'] = pd.cut(df_merged['word_count'], bins=bins, labels=labels, right=False)

In [ ]:
df_merged = df_merged.rename(columns={
    'number': 'topic_big_analysed',
    'name': 'name_big',
    'thoughts': 'thoughts_big'
})

In [ ]:
df_merged = df_merged.rename(columns={
    'name_big': 'topicname_big'
})

In [ ]:
print(df_merged.columns.tolist())

In [ ]:
df_merged.to_csv("bigcsvwithtopicsanalysed.csv", index=False)